In [1]:
import os
import requests

In [4]:
# Only once -- download the tiny shakespeare dataset:

# input_file_path = os.path.join(os.getcwd(), 'input.txt')
# if not os.path.exists(input_file_path):
#     data_url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
#     with open(input_file_path, 'w', encoding='utf-8') as f:
#         f.write(requests.get(data_url).text)

In [5]:
# Read and inspect the data:
with open('input.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()

In [6]:
print('Length of the dataset in characters:', len(text))
# print(text[:1000])

Length of the dataset in characters: 1115394


In [17]:
# Character list:
chars = sorted(list(set(text))) # Sort a list of characters that appear in the text: [, . & .' etc]
vocab_size = len(chars)

print(''.join(chars))
print('Vocabulary size:', vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocabulary size: 65


In [19]:
# Tokenize: convert raw text (string) to sequence of integers according to some vocabulary of possible elements. 
# Here we translate individual characters (e.g., &) into integers. 

# Mapping (look-up table) from characters to integers: iterate over all characters and c
stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}

# Encode 
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [20]:
print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there
